# Workbook explores results of the TV Test by price points

### Load packages

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# Load core dataset for ASINs 
df = pd.read_csv("Z://00_ETL/TV_test/asin_in_line_wDashboard_4.txt", sep="\t", encoding = "ISO-8859-1")

In [3]:
print(df.shape,df['week_ending'].min(),df['week_ending'].max())

(1881683, 19) 2017-11-04 00:00:00 2018-06-09 00:00:00


In [4]:
def initial_setup(df,content):
    """function makes basic data type transformations, 
    converts object to datetime format and renames lifecycles, gets month"""
    
    
    df['week_ending'] = pd.to_datetime(df['week_ending'])
    df['release_date'] = pd.to_datetime(df['release_date'])
    df['month'] = df['week_ending'].dt.month
    
    #Subset and work only with particular content    
    df = df[df['content'] == content]
    
        
    
    #Add only last lifecycle in the promo period to be the base, rest get last content age
    age = df[(df['week_ending'] >= '2018-03-10')
                       & (df['week_ending'] < '2018-05-12')].groupby("asin")['content_age'].first().reset_index()
    
    age.columns = ['asin','lifecycles_last']
    
    
    merge_age = pd.merge(left = df, right = age, left_on = "asin", right_on = "asin", how = "left")
    
    add_on = merge_age[merge_age['lifecycles_last'].isnull()].groupby("asin")['content_age'].last().reset_index()
    add_on.columns = ['asin','add_on']
    
    df_next = pd.merge(left = merge_age, right = add_on, left_on = "asin", right_on = "asin", how = "left")
    
    
    df_next['lifecycles_last'] = np.where(df_next['lifecycles_last'].isnull(), df_next['add_on'],df_next['lifecycles_last'])
    df_next = df_next.drop(['add_on'], axis = 1)
    
    df_next['easy_lifecycles'] = np.where(df_next['lifecycles_last'].isin(['New Release','Recent release']),'RR','Catalogue')
    
    
    return df_next

In [5]:
df = initial_setup(df, "Season")

In [6]:
print(df.shape, df['asin'].nunique())
df.head()

(244190, 22) 13488


,week_ending,is_promo,asin,is_promo_asin,content,title_name,studio_name,contract_id,release_date,is_hd,...,content_age,customers,new_customers,units,ops,cogs,ppm,month,lifecycles_last,easy_lifecycles
0,2017-11-04,non_promo_period,B00HXG8PQK,non_promo_asins,Season,Yakari - Staffel 2,"Ki.Ka, ZDF, ORF",UXKN4,2010-07-05,N,...,Deep Catalogue,34,1,34,276.82,185.86,0.3285,11,Deep Catalogue,Catalogue
1,2017-11-04,non_promo_period,B00QX1DHFU,non_promo_asins,Season,Phineas und Ferb Staffel 1,Disney Channel,UXDY4,2014-02-26,Y,...,Deep Catalogue,5,1,5,60.37,42.34,0.2986,11,Deep Catalogue,Catalogue
2,2017-11-04,non_promo_period,B071FN9YNC,Promo_Asins,Season,München 7,ARD,UXTP4,2017-06-02,Y,...,Recent release,1,0,1,12.60,8.82,0.3000,11,6mo_1yr_Catalogue,Catalogue
3,2017-11-04,non_promo_period,B0757S91QL,non_promo_asins,Season,"Looney Tunes: Best Of Tweety und Sylvester, Vo...",Warner Bros.,UXWB4,2017-04-10,N,...,6mo_1yr_Catalogue,3,0,3,37.80,22.92,0.3936,11,1_2yr_Catalogue,Catalogue
4,2017-11-04,non_promo_period,B01F2JHD1C,Promo_Asins,Season,Keeping Up With the Kardashians - Season 12 [OV],E!,UXNB4,2016-05-03,Y,...,1_2yr_Catalogue,2,0,2,27.72,19.98,0.2792,11,2_3yr_Catalogue,Catalogue


### STEP 1 : Critical part - sanity check to mare sure the same data as in the dashboard!

In [7]:
print("Average OPS in Feb")
df[(df['week_ending'] >= '2018-01-27')
   & (df['week_ending'] < '2018-03-10')
   & (df['is_promo_asin'] == "Promo_Asins")].groupby('week_ending')['ops'].sum().reset_index()['ops'].mean()

Average OPS in Feb


255112.76166666698

In [8]:
print(df[(df['is_promo'] == 'promo_period')
   & (df['is_promo_asin'] == "Promo_Asins")].groupby("week_ending")['new_customers',
                                                                    'units','ops'].sum().reset_index()['ops'].sum())
df[(df['is_promo'] == 'promo_period')
   & (df['is_promo_asin'] == "Promo_Asins")].groupby("week_ending")['new_customers',
                                                                    'units','ops'].sum().reset_index()


2247250.7100000205


,week_ending,new_customers,units,ops
0,2018-03-10,2077,52742,337491.67
1,2018-03-17,2106,43532,259809.48
2,2018-03-24,1895,40603,237363.10
3,2018-03-31,1842,44182,254590.04
4,2018-04-07,1634,37775,214874.27
5,2018-04-14,1172,32413,187994.60
6,2018-04-21,1001,29440,174938.32
7,2018-04-28,1109,30392,182927.48
8,2018-05-05,1214,34075,205540.21
9,2018-05-12,909,27519,191721.54


# Distribution of promo OPS by price points

In [9]:
def dist_by_revenue(df):
    t = pd.DataFrame(df[(df['is_promo'] == 'promo_period')
                        & (df['is_promo_asin'] == "Promo_Asins")].groupby("display_price")['ops'].sum().reset_index())
    t.columns = ['display_price','ops']
    t['share'] = t['ops'] / t['ops'].sum()
    t['cumsum'] = t['share'].cumsum()
    return t

In [10]:
rev_check_price_point = dist_by_revenue(df)

In [11]:
print(rev_check_price_point['ops'].sum())
round(rev_check_price_point['ops'].sum(),2) == round(df[(df['is_promo'] == 'promo_period')
   & (df['is_promo_asin'] == "Promo_Asins")].groupby("week_ending")['new_customers',
                                                                    'units','ops'].sum().reset_index()['ops'].sum(),2)

2247250.7099999427


True

In [12]:
#Check the share of OPS covered by those 3 price points
checker = rev_check_price_point[rev_check_price_point['display_price'].isin([4.99, 9.99, 14.99])]['ops'].sum() / rev_check_price_point['ops'].sum()
print("Share of the price points, ca. :" , round(checker,3) )
print("Revenue from three price points with non cleaned ASINs: ", rev_check_price_point[rev_check_price_point['display_price'].isin([4.99, 9.99, 14.99])]['ops'].sum())

Share of the price points, ca. : 0.929
Revenue from three price points with non cleaned ASINs:  2087161.9999999418


# Make a list of seasonal coefficiens

In [13]:
new_data_daily = pd.read_csv("../TV_test/data/forecast_DE_lifecycles_daily.txt", sep="\t",
                       names = ['promo_period','week_ending','transaction_date','promo_status','content','lifecycle',
                        'total_customers','new_customers','units','ops','cogs'])

In [14]:
def make_prep_for_lifecycles(df, dimension1, dimension2):
    """function makes key preparations"""
    
    df = df.dropna()
    df.is_copy = False #to kill warning message about setting a copy
    
    df[dimension1] = pd.to_datetime(df[dimension1])
    df[dimension2] = pd.to_datetime(df[dimension2])
    df['easy_lifecycles'] = np.where(df['lifecycle'].isin(['New Release','Recent release']),'RR','Catalogue')
    
    df = df[df['content'].isin(['Season'])]
    df = df[df[dimension1] < '2018-06-01']
    
    summed = pd.DataFrame(df.groupby([dimension1,'easy_lifecycles'])['ops'].sum().reset_index())
    pivoted = pd.DataFrame(pd.pivot_table(summed, values='ops', columns='easy_lifecycles', index=dimension1).reset_index())
    pivoted = pivoted.fillna(0)
    return pivoted

In [15]:
new_data_daily.head()

,promo_period,week_ending,transaction_date,promo_status,content,lifecycle,total_customers,new_customers,units,ops,cogs
0,non_promo_period,2013-09-07 00:00:00,2013-09-05,non_promo_asins,Movie,New Release,1,1,1,7.99,NaN
1,non_promo_period,2013-09-07 00:00:00,2013-09-06,non_promo_asins,Movie,6mo_1yr_Catalogue,1,0,1,11.00,NaN
2,non_promo_period,2013-09-14 00:00:00,2013-09-09,non_promo_asins,Movie,6mo_1yr_Catalogue,1,1,1,11.99,NaN
3,non_promo_period,2013-09-28 00:00:00,2013-09-24,non_promo_asins,Movie,Deep Catalogue,1,0,1,11.99,NaN
4,non_promo_period,2013-10-26 00:00:00,2013-10-22,non_promo_asins,Movie,6mo_1yr_Catalogue,1,1,2,15.12,NaN


In [16]:
def prep_seasonality_monthly(df):
    """prepares seasonality adjustments and calls another function"""
    
    prep = make_prep_for_lifecycles(df,"transaction_date","transaction_date")
    
    prep['total'] = prep['Catalogue'] + prep['RR']
    prep['month'] = prep['transaction_date'].dt.month
    prep['year']  = prep['transaction_date'].dt.year
    #prep['mnth_yr'] = prep['transaction_date'].apply(lambda x: x.strftime('%B-%Y'))   
    
    t = pd.DataFrame(prep.groupby(['month','year'])['Catalogue','RR','total'].sum().reset_index())
    t = t[t['year'] >= 2015]
    
    t = t.sort_values(['year','month'], ascending = [True,True]).reset_index()
    t = t.drop('index', axis = 1)
    
    t['total_new'] = np.where((t['year'] == 2017) & (t['month'] ==7), t[(t['year'] == 2017)
                                        & (t['month'] < 7 )]['total'].mean(), t['total'])
    t['RR_new'] = np.where((t['year'] == 2017) & (t['month'] ==7), t[(t['year'] == 2017)
                                        & (t['month'] < 7)]['RR'].mean(), t['RR'])
    t['Catalogue_new'] = np.where((t['year'] == 2017) & (t['month'] ==7), t[(t['year'] == 2017)
                                        & (t['month'] < 7)]['Catalogue'].mean(), t['Catalogue'])

    
    return t

In [17]:
def combine_seasonality():
    """function provides for every month a seasonality factor"""
    
    def get_shape(df,dimension):  
        monthly_mean = df[df['year'] >= 2017].groupby("month")[dimension].mean()
        all_mean = df[df['year'] >= 2017][dimension].mean()
        l = pd.DataFrame({"monthly_mean":monthly_mean,"all_mean":all_mean}).reset_index()
        l['factor'] = l['monthly_mean'] / l['all_mean']
        l = l.drop(['all_mean',"monthly_mean"], axis = 1)
        l.columns = ['month', dimension]
        return l
    
    l1 = get_shape(prep_seasonality_monthly(new_data_daily),"total_new")
    l2 = get_shape(prep_seasonality_monthly(new_data_daily),"RR_new")
    l3 = get_shape(prep_seasonality_monthly(new_data_daily),"Catalogue_new")
    
    g1 = pd.merge(left=l1,right=l2,left_on="month",right_on="month",how='left')
    g2 = pd.merge(left=g1,right=l3,left_on="month",right_on="month", how = "left")
    
    g2.columns = ['month','total','RR','Catalogue']
    return g2

In [18]:
sindex = combine_seasonality()

In [19]:
sindex

,month,total,RR,Catalogue
0,1,1.129071,1.068263,1.165034
1,2,0.967600,1.006449,0.944624
2,3,0.954687,0.898981,0.987632
3,4,0.882711,0.828018,0.915057
4,5,0.894475,0.825964,0.934993
5,6,0.799144,0.879082,0.751869
6,7,0.796890,0.777546,0.808330
7,8,1.265205,1.408715,1.180332
8,9,1.155634,1.227559,1.113097
9,10,1.162811,1.258844,1.106017


In [20]:
def get_melt(df):
    """function transforms the format from wide to long"""
    t = pd.melt(sindex, id_vars=['month'])
    t.columns = ['month','easy_lifecycles','factor']
    return t

In [21]:
sindex_melt = get_melt(sindex)

In [22]:
sindex_melt.head()

,month,easy_lifecycles,factor
0,1,total,1.129071
1,2,total,0.967600
2,3,total,0.954687
3,4,total,0.882711
4,5,total,0.894475


# Get ASINs in promo at these price points in the non promo period

In [23]:
def get_asin_at_price_points(df):
    """function selects asins that had certain price points
    and selects from pre period the same ones to make proper comparison"""
    asins_in_the_list = df[(df['is_promo'] == 'promo_period')
                       & (df['is_promo_asin'] == 'Promo_Asins')
                       & df['display_price'].isin([4.99,9.99,14.99])][['asin','display_price']]
    
    
    asins_in_shape = asins_in_the_list.groupby("asin")['display_price'].min().reset_index()
    
    #To make proper comparison get an array of those above in the non promo period
    asins_in_non_promo = pd.DataFrame(df[(df['is_promo'] == 'non_promo_period')
                            & (df['asin'].isin(asins_in_shape['asin']))]['asin'].unique(), columns = ['asin'])
    
    
    #Subset from all only those that are also in non promo
    final_list = pd.merge(left = asins_in_non_promo, right=asins_in_shape,left_on="asin",right_on="asin",how="left")
    
    
    new_df = df[df['asin'].isin(final_list['asin'])]
    
    new_new = pd.merge(left=new_df,right=final_list,left_on="asin",right_on="asin",how='left')
    new_new.rename(columns = {"display_price_x":"original_price","display_price_y":"price_in_promo"}, inplace=True)
    return new_new


In [24]:
new_df = get_asin_at_price_points(df)
new_df.shape

(56729, 23)

### Step 2 - Critical - get same number of asins in promo and non promo

In [25]:
print(new_df[(new_df['is_promo'] == "non_promo_period")]['asin'].nunique())
print(new_df[(new_df['is_promo'] == "promo_period")]['asin'].nunique())

2474
2474


In [26]:
print("Revenue from promo period :", new_df[new_df['is_promo'] == 'promo_period']['ops'].sum())
print("Period contained:", new_df['is_promo'].unique())
print("Price points:", new_df['price_in_promo'].unique())

Revenue from promo period : 2240613.049999834
Period contained: ['non_promo_period' 'promo_period']
Price points: [  4.99  14.99   9.99]


# Apply seasonality factors

In [27]:
def seasonality_factors(df1,df2):
    """function merges seasonality factors and the output from get_asin_at_price_points"""
    output = pd.merge(left=df1,right=df2,
                      left_on=['month','easy_lifecycles'],
                      right_on=['month','easy_lifecycles'],how='left')
    
    return output

In [28]:
new_df_seas = seasonality_factors(new_df,sindex_melt)
new_df_seas.shape

(56729, 24)

### Check the share of cumulative price points in the total Promo OPS

In [29]:
def prep_dataset_dimension(df, dimension = None):
    """adds complexity to the analysis in the dimension using optional argument"""
    
    #Notice: df will be updated here so export after the function will contain both of these columns
    df['ops_deseas']           = df['ops'] / df['factor']
    df['units_deseas']         = df['units'] / df['factor']
    
    
    #use optional agrument
    
#     if dimension is None:
#         a = pd.DataFrame(df.groupby(["is_promo","asin"])['week_ending'].size().reset_index())
#     else: 
#         a = pd.DataFrame(df.groupby(["is_promo","asin", dimension])['week_ending'].size().reset_index())
#     key_metrics = pd.DataFrame(df.groupby(["is_promo","asin"])['ops','ops_deseas',
#                                                                'units','units_deseas'].sum().reset_index())
    
#     m = pd.merge(left=key_metrics, right=a, left_on=['is_promo','asin'], right_on=['is_promo','asin'], how = 'left')
#     m['ops_per_week'] = m['ops'] / m['week_ending']
#     m['ops_deseas_per_week'] = m['ops_deseas'] / m['week_ending']
    
#     m['units_per_week'] = m['units'] / m['week_ending']
#     m['units_deseas_per_week'] = m['units_deseas'] / m['week_ending']
    
    
    
    
    before = pd.DataFrame(df[df['is_promo']=='non_promo_period'].
                          groupby(["asin"])["original_price"].mean().reset_index())
    
       
    key_out = pd.merge(left=df,right=before,left_on=['asin'],
                       right_on=['asin'],how="left")
    
    key_out.rename(columns={"original_price_x":"original_price","original_price_y":"mean_non_promo_price"}, inplace = True)

    return key_out

In [30]:
new_df_seas_wbefore = prep_dataset_dimension(new_df_seas)
new_df_seas_wbefore.shape

(56729, 27)

In [31]:
new_df_seas_wbefore.columns

Index(['week_ending', 'is_promo', 'asin', 'is_promo_asin', 'content',
       'title_name', 'studio_name', 'contract_id', 'release_date', 'is_hd',
       'original_price', 'season_number', 'content_age', 'customers',
       'new_customers', 'units', 'ops', 'cogs', 'ppm', 'month',
       'lifecycles_last', 'easy_lifecycles', 'price_in_promo', 'factor',
       'ops_deseas', 'units_deseas', 'mean_non_promo_price'],
      dtype='object')

In [33]:
new_df_seas_wbefore[(new_df_seas_wbefore['week_ending'] >= '2018-01-27')
                    & (new_df_seas_wbefore['week_ending'] <= '2018-03-03')]['ops'].sum()/6*10

2541510.016666679

In [34]:
new_df_seas_wbefore[(new_df_seas_wbefore['is_promo'] == 'promo_period')]['ops'].sum()

2240613.049999834

In [36]:
df1 = new_df_seas_wbefore[(new_df_seas_wbefore['week_ending'] >= '2018-01-27')
                          & (new_df_seas_wbefore['week_ending'] <= '2018-03-03')]
df2 = new_df_seas_wbefore[(new_df_seas_wbefore['is_promo'] == 'promo_period')]

frames = [df1,df2]
gluon = pd.concat(frames)

In [37]:
gluon.columns

Index(['week_ending', 'is_promo', 'asin', 'is_promo_asin', 'content',
       'title_name', 'studio_name', 'contract_id', 'release_date', 'is_hd',
       'original_price', 'season_number', 'content_age', 'customers',
       'new_customers', 'units', 'ops', 'cogs', 'ppm', 'month',
       'lifecycles_last', 'easy_lifecycles', 'price_in_promo', 'factor',
       'ops_deseas', 'units_deseas', 'mean_non_promo_price'],
      dtype='object')

In [64]:
factor = 5/3

def get_output_dimension(df,dimension1 = None, dimension2 = None):
    """make first output with 2 optional arguments"""
    if dimension1 is None and dimension2 is None:
        get = df.groupby(["is_promo"])['ops','ops_deseas','units','units_deseas','new_customers'].sum().reset_index()
    
    elif dimension1 is not None and dimension2 is None:
        get = df.groupby(["is_promo", dimension1])['ops','ops_deseas',
                                                  'units','units_deseas','new_customers'].sum().reset_index()
    
    else:
        get = df.groupby(["is_promo", dimension1, dimension2])['ops','ops_deseas',
                                                  'units','units_deseas','new_customers'].sum().reset_index()
  
    
    
    get.set_index("is_promo", inplace=True)
       
    get.loc["non_promo_period",['ops',"ops_deseas","units","units_deseas","new_customers"]] *= factor 
    
    get = get.reset_index()
    return get        

In [67]:
get = get_output_dimension(gluon,"price_in_promo","lifecycles_last")

In [68]:
get

,is_promo,price_in_promo,lifecycles_last,ops,ops_deseas,units,units_deseas,new_customers
0,non_promo_period,4.99,1_2yr_Catalogue,227500.716667,231124.515010,17776.666667,18001.407535,695.000000
1,non_promo_period,4.99,2_3yr_Catalogue,232090.933333,236438.278294,19775.000000,20126.171813,808.333333
2,non_promo_period,4.99,6mo_1yr_Catalogue,107900.350000,109927.171981,8135.000000,8286.018538,233.333333
3,non_promo_period,4.99,Deep Catalogue,568420.683333,578934.409622,47183.333333,47909.056835,1766.666667
4,non_promo_period,4.99,New Release,443.216667,447.360725,33.333333,33.713627,1.666667
5,non_promo_period,4.99,Recent release,57933.650000,58046.256643,5433.333333,5434.905193,121.666667
6,non_promo_period,9.99,1_2yr_Catalogue,203057.316667,205846.542826,12698.333333,12764.321320,1183.333333
7,non_promo_period,9.99,2_3yr_Catalogue,94745.333333,96247.715740,6500.000000,6600.771840,151.666667
8,non_promo_period,9.99,6mo_1yr_Catalogue,205023.816667,208276.531444,12430.000000,12601.010869,1058.333333
9,non_promo_period,9.99,Deep Catalogue,276364.733333,281498.599015,19425.000000,19773.653380,365.000000


In [73]:
gluon.groupby(['price_in_promo',"lifecycles_last"])['asin'].nunique()

price_in_promo  lifecycles_last  
4.99            1_2yr_Catalogue      480
                2_3yr_Catalogue      443
                6mo_1yr_Catalogue    128
                Deep Catalogue       618
                New Release            2
                Recent release        59
9.99            1_2yr_Catalogue      112
                2_3yr_Catalogue      124
                6mo_1yr_Catalogue     80
                Deep Catalogue       245
                New Release            3
                Recent release        41
14.99           1_2yr_Catalogue       39
                2_3yr_Catalogue       25
                6mo_1yr_Catalogue     35
                Deep Catalogue        13
                Recent release        27
Name: asin, dtype: int64